In [1]:
import geopandas as gpd
import pandas as pd
from siuba import *

from tqdm.notebook import tqdm
from calitp_data_analysis import utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [43]:
crow_folder = 'gs://calitp-analytics-data/data-analyses/py_crow_flies/'

In [3]:
# path = f'{crow_folder}CentralCal_POIs.zip'

# Datasets:

* CentralCal_POIs
* NorCal_POIs
* Mojave_POIs
* SoCal_POIs

In [45]:
# Read in Shapefile of grid (or origin) points. Points must have two required columns:
  #1. grid_code: The value of the opportunities being measured. In this case, the number of opportunities within the grid cell.
  #2. Point_ID: A unique id for each grid in character format.
central = gpd.read_file('./CA_Points.zip!CentralCal_POIs.shp')

In [46]:
utils.geoparquet_gcs_export(central, crow_folder, 'CentralCal_POIs')

In [47]:
nor = gpd.read_file('./CA_Points.zip!NorCal_POIs.shp')

In [48]:
utils.geoparquet_gcs_export(nor, crow_folder, 'NorCal_POIs')

In [50]:
so = gpd.read_file('./CA_Points.zip!SoCalPOIs.shp')

In [51]:
utils.geoparquet_gcs_export(so, crow_folder, 'SoCal_POIs')

In [52]:
mo = central = gpd.read_file('./CA_Points.zip!Mojave_POIs.shp')

In [53]:
utils.geoparquet_gcs_export(mo, crow_folder, 'Mojave_POIs')

In [5]:
# Transform the grid points to your preferred CRS
central = central.to_crs(shared_utils.geography_utils.CA_NAD83Albers).set_index('pointid')
central = central >> select(-_.Point_ID)

In [6]:
central.crs

<Derived Projected CRS: EPSG:3310>
Name: NAD83 / California Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - California.
- bounds: (-124.45, 32.53, -114.12, 42.01)
Coordinate Operation:
- name: California Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [7]:
# Create a SF dataset for destination points by filtering grid points to only those with opportunities > 0 (this reduces computing time)
dest_points = central >> filter(_.grid_code > 0)

In [8]:
dest_stripped = dest_points >> select(_.geometry)

In [9]:
dest_stripped

,geometry
pointid,
736,POINT (-154261.074 181991.837)
2280,POINT (-324523.035 186771.201)
4547,POINT (-324532.413 186536.371)
9509,POINT (-223766.167 182667.333)
9950,POINT (-119884.117 180472.122)
...,...
3930582,POINT (118932.954 -236728.511)
3931842,POINT (-130494.782 -236807.131)
3932635,POINT (65930.463 -237548.783)


## Look up all distances in advance?

In [10]:
import numpy as np

In [12]:
origins = central

In [37]:
origins_stripped = origins.reset_index() >> select(-_.grid_code)

In [38]:
origins_stripped 

,pointid,geometry
0,1,POINT (-327338.431 187119.328)
1,2,POINT (-327103.035 187109.849)
2,3,POINT (-326867.638 187100.376)
3,4,POINT (-326632.241 187090.910)
4,5,POINT (-326396.844 187081.451)
...,...,...
2850974,3935450,POINT (201661.211 -235671.905)
2850975,3935451,POINT (201908.855 -235666.059)
2850976,3935452,POINT (202156.499 -235660.207)
2850977,3935453,POINT (202404.143 -235654.347)


In [39]:
result = {}

def create_od(row):
    global result
    # print(row)
    # print(row.Name)
    result[row.pointid] = dest_stripped.distance(row.geometry)
    return

In [41]:
origins_stripped[:100].apply(create_od, axis = 1)

0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Length: 100, dtype: object

In [42]:
result

{1: pointid
 736        173153.292975
 2280         2836.837913
 4547         2865.933776
 9509       103667.903326
 9950       207560.781353
                ...      
 3930582    615471.478027
 3931842    467398.187012
 3932635    578794.806313
 3934901    578810.708901
 3935108    614401.484836
 Length: 29611, dtype: float64,
 2: pointid
 736        172917.719146
 2280         2602.130344
 4547         2633.813300
 9509       103432.317031
 9950       207325.202155
                ...      
 3930582    615294.286255
 3931842    467290.499509
 3932635    578627.932168
 3934901    578643.935254
 3935108    614224.733103
 Length: 29611, dtype: float64,
 3: pointid
 736        172682.145281
 2280         2367.598178
 4547         2402.369169
 9509       103196.730758
 9950       207089.622919
                ...      
 3930582    615117.138170
 3931842    467182.912056
 3932635    578461.110650
 3934901    578477.214292
 3935108    614048.025393
 Length: 29611, dtype: float64,
 4: pointi

In [16]:
dest_stripped.distance(origins.geometry.iloc[0])

pointid
736        173153.292975
2280         2836.837913
4547         2865.933776
9509       103667.903326
9950       207560.781353
               ...      
3930582    615471.478027
3931842    467398.187012
3932635    578794.806313
3934901    578810.708901
3935108    614401.484836
Length: 29611, dtype: float64

## Buffering and so

In [39]:
buffer_mi = 20
buffer_m = buffer_mi * 1609.34

In [38]:
origins = central[:100].copy()

In [40]:
origins_joined = origins.sjoin(dest_points, how='left', predicate='intersects',
                              rsuffix='dest')

In [41]:
origins_joined

,pointid_left,grid_code_left,Point_ID_left,geometry,index_dest,pointid_dest,grid_code_dest,Point_ID_dest
0,1,0.0,id_1,"POLYGON ((-295151.631 187119.328, -295306.620 ...",190638,299245,1.0,id_299245
0,1,0.0,id_1,"POLYGON ((-295151.631 187119.328, -295306.620 ...",178747,281110,1.0,id_281110
0,1,0.0,id_1,"POLYGON ((-295151.631 187119.328, -295306.620 ...",178746,281109,1.0,id_281109
0,1,0.0,id_1,"POLYGON ((-295151.631 187119.328, -295306.620 ...",171313,269776,1.0,id_269776
0,1,0.0,id_1,"POLYGON ((-295151.631 187119.328, -295306.620 ...",162393,256175,1.0,id_256175
...,...,...,...,...,...,...,...,...
99,100,0.0,id_100,"POLYGON ((-271846.090 186213.939, -272001.078 ...",57349,93089,1.0,id_93089
99,100,0.0,id_100,"POLYGON ((-271846.090 186213.939, -272001.078 ...",54383,88433,1.0,id_88433
99,100,0.0,id_100,"POLYGON ((-271846.090 186213.939, -272001.078 ...",17978,29485,1.0,id_29485
99,100,0.0,id_100,"POLYGON ((-271846.090 186213.939, -272001.078 ...",2766,4547,1.0,id_4547
